In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

from deep_fib.sci_net import SCIBlockCfg, SCINet
from deep_fib.data import DeepFIBDataset, get_masks
from deep_fib.core import DeepFIBEngine

from utils.data import Marconi100Dataset, get_dataset_paths
from utils.training import training_loop
from utils.summary import SummaryWriter

In [3]:
paths = get_dataset_paths("../data")
train, test = train_test_split(paths, test_size=0.1, random_state=42)


In [4]:
m_data_train = Marconi100Dataset(train, scaling="minmax")
m_data_test = Marconi100Dataset(test, scaling="minmax")


Loading: 100%|██████████| 25/25 [01:07<00:00,  2.70s/it]


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

horizon = 1024
stride = 1000
n_masks = 100

batch_size = 32

num_encoder_levels = 2

log_dir = "./deep_fib_outs"
lr = 1e-3
num_epochs = 5
step_size = 2

hidden = None
block_cfg = SCIBlockCfg(input_dim=460, hidden_size=4, kernel_size=3, dropout=0.5,)

anomaly_threshold = 0.1 # to be tuned


cuda


In [6]:
dataset_train = DeepFIBDataset(
    m_data_train, horizon=horizon, stride=stride
)
dataset_test = DeepFIBDataset(m_data_test, horizon=horizon, stride=stride)

masks = get_masks(horizon, n_masks).float()

len(dataset_train), len(dataset_test), masks.size()


(3049, 353, torch.Size([100, 1024, 460]))

In [7]:
train_loader = DataLoader(
    dataset_train,
    batch_size,
    shuffle=True,
)
test_loader = DataLoader(
    dataset_test,
    batch_size,
    shuffle=False,
)
len(train_loader), len(test_loader)


(96, 12)

In [8]:
model = SCINet(
    output_len=horizon,
    input_len=horizon,
    num_encoder_levels=num_encoder_levels,
    hidden_decoder_sizes=hidden,
    block_config=block_cfg,
).float()

engine = DeepFIBEngine(anomaly_threshold, masks)

optim = Adam(model.parameters(), lr=lr)
lr_sched = CosineAnnealingLR(optim, num_epochs)

In [9]:
with SummaryWriter(log_dir) as writer:
    training_loop(
        model=model,
        engine=engine,
        num_epochs=num_epochs,
        train_dataloader=train_loader,
        test_dataloader=test_loader,
        device=device,
        optimizer=optim,
        lr_scheduler=lr_sched,
        writer=writer,
        save_path=log_dir + "/models",
    )


Epoch 0 - train_loss = 0.082 - test_loss = 0.050 - lr = 1.00e-03


Epoch 1 - train_loss = 0.044 - test_loss = 0.039 - lr = 9.05e-04


Epoch 2 - train_loss = 0.038 - test_loss = 0.034 - lr = 6.55e-04


Epoch 3 - train_loss = 0.035 - test_loss = 0.032 - lr = 3.45e-04


Epoch 4 - train_loss = 0.033 - test_loss = 0.031 - lr = 9.55e-05
